In [8]:
# Jellyfish file parsing
# fasta file from jellyfish dump
# Format example: this k-mer is present 4 times
# >4
# TATAGCATTGAAACTAACCCAAGTAAGTGAA
filename = "data/test.txt"
kmer_table = {}
with open(filename) as f:
    lines = f.readlines()
    i = 0
    new = True
    for line in lines:
        if i % 2 == 0:
            count = int(line[1:])
        else:
            kmer = line[:-1]
            kmer_table[kmer] = count
        i += 1

In [ ]:
# Data processing
print(kmer_table)

In [ ]:
# Neural network architecture

In [ ]:
# Model training

In [ ]:
# Model testing